<a href="https://colab.research.google.com/github/PaulToronto/DataCamp-Track---Machine-Learning-Scientist-in-Python/blob/main/6_4_Machine_Learning_with_Tree_Based_Models_in_Python_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6-4 Machine Learning with Tree-Based Models in Python - Boosting

## Imports

In [1]:
import pandas as pd

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Data

In [2]:
base_url = 'https://drive.google.com/uc?id='

### Wisconsin Breast-Cancer Dataset

In [3]:
id = '1oqwkLiOXsHomv_Nhm4JhEUf0GQE8h1rp'
breast = pd.read_csv(base_url + id)
breast = breast.drop(['id', 'Unnamed: 32'], axis=1)
breast['diagnosis'] = (breast['diagnosis'] == 'M').astype(int)
breast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

### Indian Liver Patient Dataset

In [4]:
id = '1ZIKZwQV88fV7RFUSkhrTbGWGBxYxp9Rh'
liver = pd.read_csv(base_url + id)
liver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


## Adaboost

### Boosting

- **Boosting** is an ensemble method in which many predictors are trained and eah predictor learns from the errors of its predecessor
- **Boosting** (more formally): Ensemble method combining several weak learners to form a strong learner
- **Weak learner**: Model doing slightly better than random guessing
  - Exammple of a weak learner: Decision stump (CART whose maximum depth is 1)
- Train an ensemble of predictors sequentially
- Each predictor tries to correct its predecessor
- Popular boosting methos explored in this course:
  - AdaBoost
  - Gradient Boosting

### What if you use strong learners?

- If you use strong learners, you could still get good performance, but boosting loses its magic.
- It would be harder to control overfitting.
- It would take longer to train (strong learners are usually more complex).
- You might need to regularize heavily (for example, using shallow trees, limiting depth, adding shrinkage/learning rate).

👉 That’s why in practical boosting methods like Gradient Boosting Machines or XGBoost, they use shallow trees (e.g., depth 3–5) as the base learners.

### AdaBoost

- Stands for **Ada**ptive **Boost**ing
- Each predictor pays more attention to the instances wrongly predicted by its predecessor
  - Achieved by changing the weights of training instances
- Each predictor is assigned a coefficient, $\alpha$
  - $\alpha$ depend's on the predictor's training error

<img src='https://drive.google.com/uc?export=view&id=101nnlow_NJd9FAetsqtxyLcs21vvMjA3'>

- First, Predictor 1 is trained on the intitial dataset, $(X,y)$
    - The training error for Predictor 1 is determined
    - This error can then be used to determine $\alpha_{1}$, wich is Predictor 1's coefficient
- $\alpha_{1}$ is then used to dtermime the weights, $W^{(2)}$ of the training instances for Predictor 2
    - Notice how the incorrectly predicted instances (shown in green) acquire higher weights
    - When weighted instances are used to train Predictor 2, this predictor is forced to pay more attention to the incorrectly predicted instances
- This process is repeated sequentially, until the $N$ predictors forming the ensemble are trained


### Learning Rate

<img src='https://drive.google.com/uc?export=view&id=1Q2UOUpPFfGQGSFUjT0hNB0H8jca7mRcD'>

- An important parameter used in training is the learning rate, $\eta$, which is between 0 and 1
- $\eta$ is used to shrink the coefficient $\alpha$ of a trained predictor
- There is a trade-off between $\eta$ and the number of estimators
    - A smaller value of $\eta$, should be compensated by a greater number of estimators

### AdaBoost: Prediction

- Classification
  - Weighted majority voting
  - In `sklearn`: `AdaBoostClassifier`
- Regression
  - Weighted average
  - In `sklearn`: `AdaBoostRegressor`
- Individual predictors need not be CARTs, but CARTs are used most commonly in boosting because of their high variance

### AdaBoost Classification in sklearn (Breast-Cancer dataset)

In [5]:
X = breast.drop('diagnosis', axis=1)
y = breast['diagnosis']

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=SEED)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

In [6]:
y_train.value_counts()

,count
diagnosis,
0,250
1,148


In [7]:
# Instantiate a classifiation tree
dt = DecisionTreeClassifier(max_depth=1, random_state=SEED)

# Instatiate an AdaBoost classifier
adab_clf = AdaBoostClassifier(
    estimator=dt,
    n_estimators=100
    # default learning_rate=1.0
)

adab_clf

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1,
                                                    random_state=1),
                   n_estimators=100)

In [8]:
# fit to the training set
adab_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1,
                                                    random_state=1),
                   n_estimators=100)

In [9]:
# predit the test set probailities of positive class
y_pred_proba = adab_clf.predict_proba(X_test)[:, 1]
y_pred_proba

array([0.31252831, 0.69937093, 0.73079839, 0.25914854, 0.19488555,
       0.18646731, 0.62441114, 0.1868014 , 0.36967934, 0.26235342,
       0.33168798, 0.27627661, 0.50078076, 0.34104086, 0.20481819,
       0.80588115, 0.38606607, 0.47489563, 0.30033296, 0.38015609,
       0.75912557, 0.35680566, 0.48251931, 0.75985749, 0.23630788,
       0.36246375, 0.27362481, 0.75380339, 0.42785663, 0.75796596,
       0.30137037, 0.27451955, 0.28483756, 0.49582039, 0.41581003,
       0.35576117, 0.27171864, 0.40252643, 0.19344355, 0.23668538,
       0.4021515 , 0.19401584, 0.74158329, 0.26774575, 0.78460711,
       0.65983035, 0.221222  , 0.67809901, 0.38191409, 0.75929706,
       0.30026042, 0.6633022 , 0.36078822, 0.24885131, 0.70109777,
       0.81045246, 0.78321946, 0.31790319, 0.79566173, 0.62233916,
       0.47346924, 0.2994392 , 0.27813112, 0.71864862, 0.21990377,
       0.23824689, 0.32751413, 0.65589167, 0.21101589, 0.79828763,
       0.75412903, 0.41661285, 0.25221401, 0.74880886, 0.66544

In [10]:
# evaluate the test-set roc_auc_score
# - can range from 0 to 1
# - 0.9 to 1.0 is excellent
# - 0.8 to 0.9 is good
# - 0.5 is random guessing
adab_clf_roc_auc_score = roc_auc_score(y_test, y_pred_proba)
adab_clf_roc_auc_score.item()

0.9856892523364486

### Define the AdaBoost classifier

In [11]:
# preprocessing
liver = liver.dropna()
liver = liver.copy()
liver['Is_male'] = (liver['Gender'] == 'Male').astype(int)
liver['Dataset'] = (liver['Dataset'] == 1).astype(int)
X = liver.drop(['Gender', 'Dataset'], axis=1)
y = liver['Dataset']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((463, 10), (116, 10), (463,), (116,))

In [13]:
# Instantiate dt
dt = DecisionTreeClassifier(max_depth=2, random_state=1)

# Instantiate ada
ada = AdaBoostClassifier(estimator=dt, n_estimators=180, random_state=1)

ada

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2,
                                                    random_state=1),
                   n_estimators=180, random_state=1)

In [14]:
ada.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2,
                                                    random_state=1),
                   n_estimators=180, random_state=1)

In [15]:
# Compute the probabilities of obtaining the positive class
y_pred_proba = ada.predict_proba(X_test)[:, 1]
y_pred_proba

array([0.71140423, 0.54065295, 0.57991289, 0.71008917, 0.60927082,
       0.66037611, 0.45595766, 0.56382698, 0.53410574, 0.50469654,
       0.72196238, 0.52084162, 0.56511164, 0.66592374, 0.5224517 ,
       0.61265821, 0.51280681, 0.67288093, 0.52025069, 0.56469861,
       0.52156516, 0.61794953, 0.57736685, 0.55107369, 0.50096233,
       0.56460195, 0.5402022 , 0.43013395, 0.56389003, 0.48514937,
       0.51119877, 0.56909178, 0.53895322, 0.49086014, 0.58015109,
       0.63654435, 0.60588982, 0.57088322, 0.5831748 , 0.49916874,
       0.67765929, 0.53975699, 0.56610979, 0.57434826, 0.52030628,
       0.56489378, 0.52252853, 0.68751351, 0.65949685, 0.51554278,
       0.58066781, 0.53097236, 0.57769292, 0.54819526, 0.62343819,
       0.51886954, 0.49817948, 0.55666844, 0.53045343, 0.49763336,
       0.49555433, 0.53996365, 0.53668678, 0.61159226, 0.5276606 ,
       0.57208751, 0.67011321, 0.68016478, 0.58073117, 0.53101619,
       0.51940123, 0.50140157, 0.47656453, 0.45490326, 0.66622

In [16]:
# Evaluate test-set roc_auc_score
ada_roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(ada_roc_auc))

ROC AUC score: 0.73


## Gradient Boosting (GB)